# Detecting Fraud in a Mobile Money Platform

Supervised Learning Capstone
<br>
Colette Gabriel

## Table of Contents:
* [Introduction to the Dataset](#data)
* [Support Vector Machine Code](#svm)
    - [SVM Test with best parameters from training](#test1)
* [SGDClassifier Code](#sgd)
    - [SGDClassifier Test with best parameters from training](#test2)

## Introduction to the Dataset  <a name="data"></a>
<br>
The dataset used in this project contains over 6 million simulated data transactions with a tiny percent (0.129%) representing fraud transactions.

Because the data is so large (the excel is nearly 1/2 GB), Github recommended against storing the Excel file there. The original data for this project can be downloaded here: 
<br><br>
Synthetic Financial Datasets For Fraud Detection
<br>
https://www.kaggle.com/ntnu-testimon/paysim1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import imblearn
from sklearn import svm
import time
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import ADASYN
%matplotlib inline

In [ ]:
filename = "/Users/colette/Documents/GitHub/credit_card_log.csv"

df = pd.read_csv(filename)
pd.options.display.float_format = '{:.3f}'.format

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
df.describe()

In [ ]:
#there are over 10 million values for the two name fields
#so for efficiency sake, we will drop -- but with more proccessing power this could be mined for useful data points
df = df.drop(['nameOrig', 'nameDest'], axis=1)

In [ ]:
#one hot encoding for 'type' feature, listing the 5 types of transactions
df = pd.concat([df, pd.get_dummies(df.type, prefix="type", drop_first=True)], axis=1)

dummy_column_names = list(pd.get_dummies(df.type, prefix="type", drop_first=True).columns)

df = df.drop('type', axis=1)

In [ ]:
df.info()

In [ ]:
df.corr()

In [ ]:
#old and new balances are very highly correlated, so drop old and just keep new balances for orig and destination
df = df.drop(['newbalanceOrig', 'newbalanceDest'], axis=1)

In [ ]:
df.info()

In [ ]:
#create filter function to filter by valid vs fraud transcations
def cat_filter(df, category, cat_filter):
    cat_filter = df.loc[df[category]== cat_filter]
    return cat_filter

In [ ]:
#examine balance of valid vs fraud transactions in the dataset
real = cat_filter(df, 'isFraud', 0)['isFraud'].count()
fake = cat_filter(df, 'isFraud', 1)['isFraud'].count()

percent_fraud = fake/(real+fake)

print('Real =', real)
print('Fake =', fake)
print('Percentage of transactions that are fraud: {:.6f}'.format(percent_fraud*100))

In [ ]:
#graphically represent the tiny percentage of fraud
names='Authentic', 'Fraud'
size=[real, fake]

fig = plt.figure()
fig.patch.set_facecolor('black')
plt.rcParams['text.color'] = 'white'

# Create a circle for the center of the plot
my_circle=plt.Circle( (0,0), 0.7, color='black')

plt.pie(size, labels=names, colors=['skyblue', 'red'])
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

In [ ]:
#step = "Maps a unit of time in the real world. In this case 1 step is 1 hour of time."
import seaborn as sns
sns.lmplot(x='step', y='amount', hue='isFraud', data=df, fit_reg=False, legend=True)
plt.title('Fraud vs. Valid Transactions by Amount')
plt.xlabel('Transactions over Time')
plt.ylabel('Amount')
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.show()

In [ ]:
#looking at variations in 'amount'
print('"Amount" stats:')
print('Min: {:.2f}'.format(df['amount'].min()))
print('Max: {:.2f}'.format(df['amount'].max()))
print('# Values: {:.0f}'.format(df['amount'].count()))

In [ ]:
#looking at variations in 'amount'
q75, q25 = np.percentile(df["amount"], [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,30,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((df["amount"] > max_val) 
                      | (df["amount"] < min_val))[0]))
    ))

In [ ]:
#looking at variations in orig balance
print('"Origination Balance" stats:')
print('Min: {:.2f}'.format(df['oldbalanceOrg'].min()))
print('Max: {:.2f}'.format(df['oldbalanceOrg'].max()))
print('# Values: {:.0f}'.format(df['oldbalanceOrg'].count()))

In [ ]:
#looking at variations in orig balance
q75, q25 = np.percentile(df["oldbalanceOrg"], [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,30,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((df["oldbalanceOrg"] > max_val) 
                      | (df["oldbalanceOrg"] < min_val))[0]))
    ))

In [ ]:
#looking at variations in dest balance
print('"Destination Balance" stats:')
print('Min: {:.2f}'.format(df['oldbalanceDest'].min()))
print('Max: {:.2f}'.format(df['oldbalanceDest'].max()))
print('# Values: {:.0f}'.format(df['oldbalanceDest'].count()))

In [ ]:
#looking at variations in dest balance
q75, q25 = np.percentile(df["oldbalanceDest"], [75 ,25])
iqr = q75 - q25

for threshold in np.arange(1,30,0.5):
    min_val = q25 - (iqr*threshold)
    max_val = q75 + (iqr*threshold)
    print("The score threshold is: {}".format(threshold))
    print("Number of outliers is: {}".format(
        len((np.where((df["oldbalanceDest"] > max_val) 
                      | (df["oldbalanceDest"] < min_val))[0]))
    ))

In [ ]:
#create new colums for log transformed values of amount, origination balance, and destination balance
#plot before and after
df['amount_log'] = np.log(df.amount+1)
df['origBalance_log'] = np.log(df.oldbalanceOrg+1)
df['destBalance_log'] = np.log(df.oldbalanceDest+1)


plt.figure(figsize=(15, 15))

# histograms of the box-cox transformed data
plt.subplot(3, 2, 1)
plt.hist(df.amount, facecolor='k', alpha=0.75)
plt.title('Histogram Amounts')
plt.ylabel('Amount')
plt.grid(True)

plt.subplot(3, 2, 2)
plt.hist(df.amount_log, facecolor='k', alpha=0.75)
plt.title('Amounts (log transformed)')
plt.ylabel('Amount')
plt.grid(True)

# histograms of the box-cox transformed data
plt.subplot(3, 2, 3)
plt.hist(df.oldbalanceOrg, facecolor='k', alpha=0.75)
plt.title('Histogram Origination Balance')
plt.ylabel('Amount')
plt.grid(True)

plt.subplot(3, 2, 4)
plt.hist(df.origBalance_log, facecolor='k', alpha=0.75)
plt.title('Origination Balance (log transformed)')
plt.ylabel('Amount')
plt.grid(True)

# histograms of the box-cox transformed data
plt.subplot(3, 2, 5)
plt.hist(df.oldbalanceDest, facecolor='k', alpha=0.75)
plt.title('Histogram Destination Balance')
plt.ylabel('Amount')
plt.grid(True)

plt.subplot(3, 2, 6)
plt.hist(df.destBalance_log, facecolor='k', alpha=0.75)
plt.title('Destination Balance (log transformed)')
plt.ylabel('Amount')
plt.grid(True)

plt.tight_layout()
plt.show()


## Undersampling Method

In [ ]:
#under sampling method - generate multiple small datasets of 16,426 transactions
#use 80% of fraud transactions (with 20% reserved as a final test set) & same number of random valid samples

fraud_count = len(df[df.isFraud == 1])
fraud_sample_count = int(len(df[df.isFraud == 1])*0.8)

#collect all valid transactions for random use in balanced datasets
valid_indices = df[df.isFraud == 0].index

#collect all fraud transactions and set 20% aside for a final test
fraud_indices = df[df.isFraud == 1].index
fraud_train = fraud_indices[:fraud_sample_count]  #6570 samples used in every training dataset
fraud_test = fraud_indices[fraud_sample_count:]   #1643 samples set aside for a final test dataset

data_num = 25
data_sample_under = dict()

#generate small random datasets for training purposes using UNDERSAMPLING
for i in range (0,data_num):
    random_ind_under = np.random.choice(valid_indices, fraud_sample_count, replace=False)  #random sample of valid indices
    under_sample_ind = np.concatenate([fraud_train, random_ind_under])
    data_sample_under[i] = df.loc[under_sample_ind]
    
#generate final testing dataset
random_ind_under = np.random.choice(valid_indices, math.ceil(fraud_count*0.2), replace=False)  #random sample of valid indices
under_sample_ind = np.concatenate([fraud_test, random_ind_under])
testing_sample = df.loc[under_sample_ind]

In [ ]:
#verify that sample sets are evenly balanced
real = cat_filter(data_sample_under[0], 'isFraud', 0)['isFraud'].count()
fake = cat_filter(data_sample_under[0], 'isFraud', 1)['isFraud'].count()
percent_fraud = fake/(real+fake)

print('Real =', real)
print('Fake =', fake)
print('Percentage of transactions that are fraud: {:.6f}'.format(percent_fraud*100))

In [ ]:
#stats for the final true test dataset (20% of the actual fraud transactions that were held in reserve)
real = cat_filter(testing_sample, 'isFraud', 0)['isFraud'].count()
fake = cat_filter(testing_sample, 'isFraud', 1)['isFraud'].count()
percent_fraud = fake/(real+fake)

print('Real =', real)
print('Fake =', fake)
print('Percentage of transactions that are fraud: {:.6f}'.format(percent_fraud*100))

In [ ]:
data_sample_under[24].info()

In [ ]:
scaling = StandardScaler()
pca = PCA()

obs_train = dict()
obs_test = dict()
matrix = dict()
report = dict()
tn = dict()
fp = dict()
fn = dict()
tp = dict()
acc = dict()
precision = dict()
recall = dict()
f1 = dict()
X = dict()
y = dict()

for i in range(0, len(data_sample_under)):
    X[i] = data_sample_under[i].drop(columns=['isFraud', 'amount', 'oldbalanceOrg', 'oldbalanceDest'])
    y[i] = data_sample_under[i]['isFraud']
    X[i] = scaling.fit_transform(X[i])

## Support Vector Machine code <a name="svm"></a>

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = LinearSVC(max_iter=2000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*(precision[i]*recall[i]))/((precision[i] + recall[i]))

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
# calculate precision-recall curve

precision, recall, _ = precision_recall_curve(y_test, y_pred)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          (total_prec*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = LinearSVC(max_iter=3000, tol=1e-5, loss='hinge')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
# calculate precision-recall curve

precision, recall, _ = precision_recall_curve(y_test, y_pred)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          (total_prec*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SVC(kernel='linear', C=1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
# calculate precision-recall curve

precision, recall, _ = precision_recall_curve(y_test, y_pred)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          (total_prec*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SVC(kernel='sigmoid')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

#parameter tuning with GridSearchCV
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001, 0.00001, 10]}

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf_grid = GridSearchCV(svm.SVC(), param_grid, verbose=1)
    clf_grid.fit(X_train, y_train)
    y_pred = clf_grid.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*(precision[i]*recall[i]))/((precision[i] + recall[i]))
    print("Dataset {} Best Parameters: {}".format(i, clf_grid.best_params_))
    print("\nDataset {} Best Estimators: {}".format(i, clf_grid.best_estimator_))
 

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SVC(C=10, cache_size=200, class_weight=None, coef0=0.0, 
              decision_function_shape='ovr', degree=3, gamma=10, kernel='rbf',
              max_iter=-1, probability=False, random_state=None, shrinking=True, tol=0.001, verbose=False)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SVC(C=10, cache_size=200, class_weight=None, coef0=0.0, 
              decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
              max_iter=-1, probability=False, random_state=None, shrinking=True, tol=0.001, verbose=False)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SVC(C=100, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape='ovr', degree=3,
              gamma=1, kernel='rbf', max_iter=-1, probability=False, random_state=None, shrinking=True,
              tol=0.001, verbose=False)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
# calculate precision-recall curve

precision, recall, _ = precision_recall_curve(y_test, y_pred)

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          (total_prec*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SVC(C=10, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape='ovr', degree=3, 
              gamma=10, kernel='rbf', max_iter=-1, probability=False, random_state=None, shrinking=True,
              tol=0.001, verbose=False)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SVC(C=100, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape='ovr', degree=3,
              gamma=1, kernel='rbf', max_iter=-1, probability=False, random_state=None, shrinking=True, tol=0.001,
              verbose=False)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = svm.SVC(gamma='scale', decision_function_shape='ovo')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = f1_score(y_test, y_pred, average='weighted')
    
    #(2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = svm.SVC(kernel='linear', C=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = svm.SVC(kernel='linear', C=1.0)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

### SVM Test with best parameters from training <a name="test1"></a>
<br>
Run the best performing support machine vector algorithms with the final reserved test data

In [ ]:
X_finalTest = testing_sample.drop(columns=['isFraud', 'amount', 'oldbalanceOrg', 'oldbalanceDest'])
y_finalTest = testing_sample['isFraud']
X_finalTest = scaling.fit_transform(X_finalTest)

In [ ]:
print('working...')
start_time = time.time()

pca.fit(X_finalTest)
obs_test = X_finalTest.shape
clf = svm.SVC(C=100, cache_size=200, class_weight=None, coef0=0.0, decision_function_shape='ovr', degree=3,
        gamma=1, kernel='rbf', probability=False, random_state=None, shrinking=True, tol=0.001,
        verbose=False)
clf.fit(X_finalTest, y_finalTest)
y_pred = clf.predict(X_finalTest)
matrix = confusion_matrix(y_finalTest, y_pred)
report = classification_report(y_finalTest, y_pred)

#extracting true_positives, false_positives, true_negatives, false_negatives
tn, fp, fn, tp = confusion_matrix(y_finalTest, y_pred).ravel()

# Model Accuracy: how often is the classifier correct?
acc = metrics.accuracy_score(y_finalTest, y_pred)
#Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
precision = tp/(tp+fp) 
#Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
recall = tp/(tp+fn) 
#F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
f1 = (2*precision*recall)/(precision + recall)

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
print("The number of observations in this final test set is {}".format(obs_test))
print("Confusion Matrix\n", matrix)
print("Classification report\n", report)
print("\nTrue Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)
print("Accuracy: ",acc)
print("Precision {:0.2f}".format(precision))
print("Recall {:0.2f}".format(recall))
print("F1 Score {:0.2f}".format(f1))
print("Sklearn F1 {:0.2f}".format(f1_score(y_finalTest, y_pred, average='macro')))
print("\n\n")

In [ ]:
print('working...')
start_time = time.time()

pca.fit(X_finalTest)
obs_test = X_finalTest.shape
clf = svm.SVC(gamma='scale', decision_function_shape='ovo')
clf.fit(X_finalTest, y_finalTest)
y_pred = clf.predict(X_finalTest)
matrix = confusion_matrix(y_finalTest, y_pred)
report = classification_report(y_finalTest, y_pred)

#extracting true_positives, false_positives, true_negatives, false_negatives
tn, fp, fn, tp = confusion_matrix(y_finalTest, y_pred).ravel()

# Model Accuracy: how often is the classifier correct?
acc = metrics.accuracy_score(y_finalTest, y_pred)
#Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
precision = tp/(tp+fp) 
#Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
recall = tp/(tp+fn) 
#F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
f1 = (2*precision*recall)/(precision + recall)

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
print("The number of observations in this final test set is {}".format(obs_test))
print("Confusion Matrix\n", matrix)
print("Classification report\n", report)
print("\nTrue Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)
print("Accuracy: ",acc)
print("Precision {:0.2f}".format(precision))
print("Recall {:0.2f}".format(recall))
print("F1 Score {:0.2f}".format(f1))
print("\n\n")

## SGDClassifier code <a name="sgd"></a>

In [ ]:
print('working...')
start_time = time.time()

#parameter tuning with GridSearchCV
param_grid = {"n_iter_no_change": [1, 5, 10],
              "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
              "penalty": ["none", "l1", "l2"]}

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf_grid = GridSearchCV(SGDClassifier(), param_grid, verbose=1)
    #clf = SGDClassifier()
    #clf_grid = GridSearchCV(clf, param_grid=param_grid)
    clf_grid.fit(X_train, y_train)
    y_pred = clf_grid.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*(precision[i]*recall[i]))/((precision[i] + recall[i]))
    print("Dataset {} Best Parameters: {}".format(i, clf_grid.best_params_))
    print("\nDataset {} Best Estimators: {}".format(i, clf_grid.best_estimator_))
 

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SGDClassifier(tol=1e-3, shuffle=True)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SGDClassifier(alpha=0.0001, average=False, class_weight=None, early_stopping=False, 
                        epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15, learning_rate='optimal',
                        loss='hinge', max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='none', power_t=0.5,
                        random_state=None, shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
                        warm_start=False)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SGDClassifier(tol=1e-3, shuffle=True, loss='log')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])
    

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

In [ ]:
print('working...')
start_time = time.time()

for i in range(0, len(X)):
    pca.fit(X[i])
    X_train, X_test, y_train, y_test = train_test_split(X[i], y[i], test_size=0.25, random_state=0)
    obs_train[i] = X_train.shape
    obs_test[i] = X_test.shape
    clf = SGDClassifier(alpha=0.01, average=False, class_weight=None, early_stopping=False, epsilon=0.1, 
                        eta0=0.0, fit_intercept=True, l1_ratio=0.15, learning_rate='optimal', loss='hinge', 
                        max_iter=1000, n_iter_no_change=1, n_jobs=None, penalty='none', power_t=0.5, 
                        random_state=None, shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
                        warm_start=False)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    matrix[i] = confusion_matrix(y_test, y_pred)
    report[i] = classification_report(y_test, y_pred)
    #extracting true_positives, false_positives, true_negatives, false_negatives
    tn[i], fp[i], fn[i], tp[i] = confusion_matrix(y_test, y_pred).ravel()
    # Model Accuracy: how often is the classifier correct?
    acc[i] = metrics.accuracy_score(y_test, y_pred)
    #Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
    precision[i] = tp[i]/(tp[i]+fp[i]) 
    #Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
    recall[i] = tp[i]/(tp[i]+fn[i]) 
    #F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
    f1[i] = (2*precision[i]*recall[i])/(precision[i] + recall[i])
    

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
total_acc = 0
total_recall = 0
total_prec = 0
total_f1 = 0

for i in range(0, len(X)):
    print("The number of observations in {} training set is {}".format(i, obs_train[i]))
    print("The number of observations in test set is {}".format(obs_test[i]))
    print("Confusion Matrix\n", matrix[i])
    print("Classification report\n", report[i])
    print("\nTrue Negatives: ",tn[i])
    print("False Positives: ",fp[i])
    print("False Negatives: ",fn[i])
    print("True Positives: ",tp[i])
    print("Accuracy: ",acc[i])
    print("Precision {:0.2f}".format(precision[i]))
    print("Recall {:0.2f}".format(recall[i]))
    print("F1 Score {:0.2f}".format(f1[i]))
    print("\n\n")
    total_acc = total_acc + acc[i]
    total_recall = total_recall + recall[i]
    total_prec = total_prec + precision[i]
    total_f1 = total_f1 + f1[i]
    
print("Ave accuracy = {:0.2f}".format((total_acc*100)/len(X)))
print("Ave recall = {:0.2f}".format((total_recall*100)/len(X)))
print("Ave precision = {:0.2f}".format((total_prec*100)/len(X)))
print("Ave F1 = {:0.2f}".format((total_f1*100)/len(X)))

### SGDClassifier Test with best parameters from training <a name="test2"></a>
<br>
Run the best performing SGD algorithms with the final reserved test data

In [ ]:
X_finalTest = testing_sample.drop(columns=['isFraud', 'amount', 'oldbalanceOrg', 'oldbalanceDest'])
y_finalTest = testing_sample['isFraud']
X_finalTest = scaling.fit_transform(X_finalTest)

In [ ]:
print('working...')
start_time = time.time()

pca.fit(X_finalTest)
obs_test = X_finalTest.shape
clf = SGDClassifier(tol=1e-3, shuffle=True)
clf.fit(X_finalTest, y_finalTest)
y_pred = clf.predict(X_finalTest)
matrix = confusion_matrix(y_finalTest, y_pred)
report = classification_report(y_finalTest, y_pred)

#extracting true_positives, false_positives, true_negatives, false_negatives
tn, fp, fn, tp = confusion_matrix(y_finalTest, y_pred).ravel()

# Model Accuracy: how often is the classifier correct?
acc = metrics.accuracy_score(y_finalTest, y_pred)
#Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
precision = tp/(tp+fp) 
#Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
recall = tp/(tp+fn) 
#F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
f1 = (2*precision*recall)/(precision + recall)

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
print("The number of observations in this final test set is {}".format(obs_test))
print("Confusion Matrix\n", matrix)
print("Classification report\n", report)
print("\nTrue Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)
print("Accuracy: ",acc)
print("Precision {:0.2f}".format(precision))
print("Recall {:0.2f}".format(recall))
print("F1 Score {:0.2f}".format(f1))
print("\n\n")

In [ ]:
print('working...')
start_time = time.time()

pca.fit(X_finalTest)
obs_test = X_finalTest.shape
clf = SGDClassifier(alpha=0.0001, average=False, class_weight=None, early_stopping=False, epsilon=0.1, eta0=0.0,
                    fit_intercept=True, l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
                    n_iter_no_change=5, n_jobs=None, penalty='none', power_t=0.5, random_state=None,
                    shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)
clf.fit(X_finalTest, y_finalTest)
y_pred = clf.predict(X_finalTest)
matrix = confusion_matrix(y_finalTest, y_pred)
report = classification_report(y_finalTest, y_pred)

#extracting true_positives, false_positives, true_negatives, false_negatives
tn, fp, fn, tp = confusion_matrix(y_finalTest, y_pred).ravel()

# Model Accuracy: how often is the classifier correct?
acc = metrics.accuracy_score(y_finalTest, y_pred)
#Precision - ‘Exactness’, ability of the model to return only relevant instances (ex. minimizing false positives)
precision = tp/(tp+fp) 
#Recall - ‘Completeness’, ability of the model to identify all relevant instances, True Positive Rate, aka Sensitivity.
recall = tp/(tp+fn) 
#F1 Score - Harmonic mean of Precision & Recall, used to indicate a balance between Precision & Recall providing each equal weightage, it ranges from 0 to 1
f1 = (2*precision*recall)/(precision + recall)

print("--- %s seconds ---" % round(time.time() - start_time, 2))

In [ ]:
print("The number of observations in this final test set is {}".format(obs_test))
print("Confusion Matrix\n", matrix)
print("Classification report\n", report)
print("\nTrue Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)
print("Accuracy: ",acc)
print("Precision {:0.2f}".format(precision))
print("Recall {:0.2f}".format(recall))
print("F1 Score {:0.2f}".format(f1))
print("\n\n")